In [1]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [ ]:
import os
print(documents)

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'), Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'), Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'), Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'), Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]


In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
import certifi
from langchain_groq import ChatGroq
os.environ['SSL_CERT_FILE'] = certifi.where()
groq_api_key = os.getenv("GROQ_API_KEY")
os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")
llm = ChatGroq(model="Llama3-8b-8192",groq_api_key=groq_api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F5DBFD8550>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F5DC06AD50>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


f:\DATA SCIENCE\Projects\Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
## vectorstore
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents,embedding=embeddings)
vectorstore

In [6]:
vectorstore.similarity_search("Cat")

[Document(id='e6d37b3d-4903-4169-8805-53b565628b63', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='ff3fdb30-26fc-40a8-859c-135e935a268e', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='3b104633-d440-4ac1-af3b-cea7ea12f971', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='9cb14de5-adc2-4d80-919a-0fec8ac45139', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [7]:
# Async query
await vectorstore.asimilarity_search("cat")

[Document(id='e6d37b3d-4903-4169-8805-53b565628b63', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='ff3fdb30-26fc-40a8-859c-135e935a268e', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='3b104633-d440-4ac1-af3b-cea7ea12f971', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='9cb14de5-adc2-4d80-919a-0fec8ac45139', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [8]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='e6d37b3d-4903-4169-8805-53b565628b63', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351057410240173),
 (Document(id='ff3fdb30-26fc-40a8-859c-135e935a268e', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740900039672852),
 (Document(id='3b104633-d440-4ac1-af3b-cea7ea12f971', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  1.5956902503967285),
 (Document(id='9cb14de5-adc2-4d80-919a-0fec8ac45139', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  1.6657923460006714)]

In [9]:
## retrievers
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])
 

[[Document(id='e6d37b3d-4903-4169-8805-53b565628b63', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='ff3fdb30-26fc-40a8-859c-135e935a268e', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [10]:
retriever =vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={"k":1}
)
retriever.batch(["cat","dog"])

[[Document(id='e6d37b3d-4903-4169-8805-53b565628b63', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='ff3fdb30-26fc-40a8-859c-135e935a268e', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
message = """
Answer this question using the provided context only

{question}

Context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([("human",message)])
rag_chain = {"context":retriever,"question":RunnablePassthrough()}|prompt|llm
response = rag_chain.invoke("tell me about dogs")
response.content

'According to the provided context, dogs are great companions, known for their loyalty and friendliness.'